In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']= '0'
import numpy as np
import keras
import pickle
import tensorflow as tf
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import time
import seaborn as sn
import pandas as pd
import h5py
import random
init_notebook_mode()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# parameters
data_path = '/content/drive/MyDrive/Colab Notebooks/Data/'
file_name = 'gnu_data.pkl'

In [ ]:
with open(data_path + file_name, 'rb') as f:
    data = pickle.load(f, encoding = 'latin1')
#data

In [ ]:
data['X']
data['lbl']

X     = []
modul = []
snr   = []

for i in range(len(data['X'])):
    x_ = data['X'][i]
    x_ = x_.transpose([1,0])
    X.append(x_)
    
    lbl = data['lbl'][i]
    modul.append(lbl[0])
    snr.append(int(lbl[1]))
X

In [ ]:
train_test_split = 0.8

random.seed(100)
random.shuffle(X)

random.seed(100)
random.shuffle(modul)

random.seed(100)
random.shuffle(snr)


X_train = np.array(X[:int(train_test_split*len(X))])
X_test  = np.array(X[int(train_test_split*len(X)):])

modul_train = modul[:int(train_test_split*len(X))]
modul_test  = modul[int(train_test_split*len(X)):]

snr_train = snr[:int(train_test_split*len(X))]
snr_test  = snr[int(train_test_split*len(X)):]
X_train.shape

(176000, 128, 2)

In [ ]:
mods = list(set(modul_train))
print(len(mods))

def to_onehot(yy):
    yy1 = np.zeros([1, len(mods)])
    yy1[0,mods.index(yy)] = 1
    return yy1

Y_train = np.zeros([len(modul_train),len(mods)])
Y_test = np.zeros([len(modul_test),len(mods)])

for i in range(len(modul_train)):
  Y_train[i,:] = to_onehot(modul_train[i])

for i in range(len(modul_test)):
  Y_test[i,:] = to_onehot(modul_test[i]) 

11


In [ ]:
# This is our input image
encoder_inputs = keras.Input(shape=(128,2,),
                                name = 'encoder_inputs')

encoder_1, state_h_1, state_c_1 = keras.layers.CuDNNLSTM(units = 32,
                                    return_sequences = True,
                                    return_state = True,
                                    name = 'encoder_1')(encoder_inputs)

drop_prob = 0.2
drop_1 = keras.layers.Dropout(drop_prob, name = 'drop_1')(encoder_1)

encoder_2, state_h_2, state_c_2 = keras.layers.CuDNNLSTM(units = 32,
                                    return_state = True,
                                    return_sequences = True,                
                                    name = 'encoder_2')(drop_1)

decoder = keras.layers.TimeDistributed(keras.layers.Dense(2),
                                          name = 'decoder')(encoder_2)

# 3 Dense layers for classification with bn
clf_dropout = 0.2

clf_dense_1 = keras.layers.Dense(units = 32,
                                    activation = tf.nn.relu,
                                    name = 'clf_dense_1')(state_h_2)

bn_1 = keras.layers.BatchNormalization(name = 'bn_1')(clf_dense_1)

clf_drop_1 = keras.layers.Dropout(clf_dropout, name = 'clf_drop_1')(bn_1)

clf_dense_2 = keras.layers.Dense(units = 16,
                                    activation = tf.nn.relu,
                                    name = 'clf_dense_2')(clf_drop_1)

bn_2 = keras.layers.BatchNormalization(name = 'bn_2')(clf_dense_2)

clf_drop_2 = keras.layers.Dropout(clf_dropout, name = 'clf_drop_2')(bn_2)

clf_dense_3 = keras.layers.Dense(units = 11,
                                    name = 'clf_dense_3')(clf_drop_2)

softmax = keras.layers.Softmax(name = 'softmax')(clf_dense_3)

model = keras.Model(inputs = encoder_inputs, outputs = [decoder, softmax])


learning_rate = 10 ** -3
lam = 0.1

# model.compile(loss='mse', optimizer='adam')
model.compile(loss = ['mean_squared_error', 'categorical_crossentropy'],
              loss_weights = [1 - lam, lam],
              metrics=['accuracy'],
              optimizer = 'Adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, 128, 2)]     0                                            
__________________________________________________________________________________________________
encoder_1 (CuDNNLSTM)           [(None, 128, 32), (N 4608        encoder_inputs[0][0]             
__________________________________________________________________________________________________
drop_1 (Dropout)                (None, 128, 32)      0           encoder_1[0][0]                  
__________________________________________________________________________________________________
encoder_2 (CuDNNLSTM)           [(None, 128, 32), (N 8448        drop_1[0][0]                     
______________________________________________________________________________________________

In [ ]:
# Set up some params 
nb_epoch = 100     # number of epochs to train on
batch_size = 128  # training batch size

filepath = '/content/drive/MyDrive/Colab Notebooks/Data/CNN-RNN.h5'
model.fit(X_train,
          [X_train,Y_train],
          batch_size=batch_size,
          epochs=nb_epoch,
          verbose=1,
          validation_data=(X_test,[X_test,Y_test]),
          callbacks = [
              keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
              keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')]
    )
# we re-load the best weights once training is finished
model.load_weights(filepath)

In [4]:
snrs= list(set(snr))
snrs.sort()

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
test_Y_i_hat[1].shape

In [ ]:
# Plot confusion matrix
acc = {}
for snr_value in snrs:

    # extract classes @ SNR
    test_X_i = X_test[np.where(np.array(snr_test)==snr_value)]
    test_Y_i = Y_test[np.where(np.array(snr_test)==snr_value)]    

    # estimate classes
    test_Y_i_hat = model.predict(test_X_i)[1]
    conf = np.zeros([len(mods),len(mods)])
    confnorm = np.zeros([len(mods),len(mods)])
    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1
    for i in range(0,len(mods)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    plt.figure()
    plot_confusion_matrix(confnorm, labels=mods, title="AE Confusion Matrix (SNR=%d)"%(int(snr_value)))
    
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print ("Overall Accuracy: ", cor / (cor+ncor))
    acc[snr_value] = 1.0*cor/(cor+ncor)

In [ ]:
acc_ = [acc[snr_value] for snr_value in snrs]

# Plot accuracy curve
plt.plot(snrs, acc_)
plt.xlabel("Signal to Noise Ratio")
plt.ylabel("Classification Accuracy")
plt.title("AE Classification Accuracy on RadioML 2016.10 Alpha")

In [ ]:
import pandas as pd
print('real')
pd.DataFrame(X_train[:,:,0]).head()

In [ ]:
print('imaginary')
pd.DataFrame(X_train[:,:,1]).head()